## 1. Import Dependencies

In [1]:
import os
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

## 2. Load the environment

In [2]:
environment_name = 'CartPole-v1'
env = gym.make(environment_name, render_mode="human")

episodes = 5
for episode in range(1,episodes + 1):
    state , info = env.reset()
    done = False
    score = 0 
    while not done:
        env.render()
        action = env.action_space.sample()
        obs, reward, finished, truncated, info = env.step(action)
        done = finished or truncated
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))    

env.close()
# env.action_space
# env.observation_space

Episode:1 Score:21.0
Episode:2 Score:21.0
Episode:3 Score:24.0
Episode:4 Score:13.0
Episode:5 Score:27.0


## 3. Train Model

In [3]:
log_path = os.path.join('Training','Logs')

In [4]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose=1,tensorboard_log = log_path)
model.learn(total_timesteps=20000)

Using cuda device
Logging to Training\Logs\PPO_12


e:\Python\Python311\Lib\site-packages\stable_baselines3\common\on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


-----------------------------
| time/              |      |
|    fps             | 776  |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 615         |
|    iterations           | 2           |
|    time_elapsed         | 6           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.007579278 |
|    clip_fraction        | 0.0791      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.687      |
|    explained_variance   | -0.00146    |
|    learning_rate        | 0.0003      |
|    loss                 | 7.94        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0115     |
|    value_loss           | 53.6        |
-----------------------------------------
----------------------------------

## 4. Save and Reload the Model

In [5]:
PPO_Path = os.path.join('Training', 'Saved Models', 'PPO_Model_CartPole')
model.save(PPO_Path)
del model
print(PPO_Path)
model = PPO.load(PPO_Path, env=env)



Training\Saved Models\PPO_Model_CartPole


## 5. Evaluation


In [6]:
ev_policy = evaluate_policy(model, env, n_eval_episodes=10, render=True)
env.close()
print(ev_policy)

e:\Python\Python311\Lib\site-packages\stable_baselines3\common\evaluation.py:70: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(
e:\Python\Python311\Lib\site-packages\stable_baselines3\common\vec_env\base_vec_env.py:259: UserWarning: You tried to call render() but no `render_mode` was passed to the env constructor.
  warnings.warn("You tried to call render() but no `render_mode` was passed to the env constructor.")


(500.0, 0.0)


## 6. Test Model

In [7]:
env = gym.make(environment_name, render_mode="human")

episodes = 5

for episode in range(1, episodes + 1):

    obs, info = env.reset()
    done = False
    score = 0

    while not done:
        env.render()

        action, _ = model.predict(obs)   # for SB3 models

        obs, reward, finished, truncated, info = env.step(action)
        done = finished or truncated

        score += reward

    print(f"Episode:{episode} Score:{score}")

#env.close()

Episode:1 Score:500.0
Episode:2 Score:500.0
Episode:3 Score:500.0
Episode:4 Score:500.0
Episode:5 Score:473.0


In [8]:
obs , info = env.reset()

In [9]:
model.predict(obs)

(array(1, dtype=int64), None)

## 7. Viewing Logs In TensorBoard

In [10]:
# tensorboard --logdir=Training/Logs/PPO_8

## 8. Adding a callback to Training Stage

In [11]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [12]:
save_path = os.path.join('Training', 'Saved Models')

In [13]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200, verbose=1)
eval_callback = EvalCallback(env,
                             callback_on_new_best=stop_callback,
                             eval_freq=10000, 
                             best_model_save_path = save_path, 
                             verbose=1)

In [14]:
model = PPO('MlpPolicy', env, verbose=1,tensorboard_log = log_path)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [15]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training\Logs\PPO_13
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 25.1     |
|    ep_rew_mean     | 25.1     |
| time/              |          |
|    fps             | 45       |
|    iterations      | 1        |
|    time_elapsed    | 45       |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 27.1        |
|    ep_rew_mean          | 27.1        |
| time/                   |             |
|    fps                  | 45          |
|    iterations           | 2           |
|    time_elapsed         | 90          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009067362 |
|    clip_fraction        | 0.103       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.00152    

## 9. Changing Policies

In [16]:
net_arch = [dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]

In [17]:
model = PPO('MlpPolicy', env, verbose=1,tensorboard_log = log_path, policy_kwargs={'net_arch': net_arch})

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


e:\Python\Python311\Lib\site-packages\stable_baselines3\common\policies.py:486: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])
  warnings.warn(


In [ ]:
model.learn(total_timesteps=10000, callback=eval_callback)

Logging to Training\Logs\PPO_14
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 22.3     |
|    ep_rew_mean     | 22.3     |
| time/              |          |
|    fps             | 46       |
|    iterations      | 1        |
|    time_elapsed    | 44       |
|    total_timesteps | 2048     |
---------------------------------


## 10. Using an Alternate Algorithm

In [ ]:
from stable_baselines3 import DQN

In [ ]:
model = DQN('MlpPolicy', env, verbose=1,tensorboard_log = log_path)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [ ]:
model.learn(total_timesteps=10000, callback=eval_callback)

Logging to Training\Logs\DQN_1
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 22       |
|    ep_rew_mean      | 22       |
|    exploration_rate | 0.916    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 46       |
|    time_elapsed     | 1        |
|    total_timesteps  | 88       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 19       |
|    ep_rew_mean      | 19       |
|    exploration_rate | 0.856    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 45       |
|    time_elapsed     | 3        |
|    total_timesteps  | 152      |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.523    |
|    n_updates        | 12       |
----------------------------------
----------------------------------
| rollout/            | 

e:\Python\Python311\Lib\site-packages\stable_baselines3\common\evaluation.py:70: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=10000, episode_reward=20.80 +/- 1.47
Episode length: 20.80 +/- 1.47
----------------------------------
| eval/               |          |
|    mean_ep_length   | 20.8     |
|    mean_reward      | 20.8     |
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    total_timesteps  | 10000    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.000132 |
|    n_updates        | 2474     |
----------------------------------
